In [24]:
import pandas as pd 
import math 

# Open excel file 
fixes = pd.read_excel('ScriptTest1_Fixes.xlsx')

# Splice for useful information
fixes = fixes.loc[:,['Initial ID','Changed ID', 'Time First Seen']].dropna()

# Sort by Time 
fixes = fixes.sort_values(by=['Time First Seen'])

graph = {}

#Construct graph 
for index, row in fixes.iterrows():
    changed_id = int(row['Changed ID'])
    initial_id = int(row['Initial ID'])
    if(changed_id in graph): 
        if(initial_id not in graph[changed_id]): 
            graph[changed_id].append(initial_id)
    else: 
        graph[changed_id] = [initial_id]






{743: [22], 722: [415], 757: [429], 714: [18], 778: [2], 742: [24], 808: [15]}